In [19]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder
import pickle
# import matplotlib.pyplot as plt
# %matplotlib inline

class encode_categorical_feature:
    def __init__(self,feature):
        self.f_set = []
        self.feature = feature
    def fit(self,data):
        if self.feature in data.columns:
            self.f_set = sorted(list(data[self.feature].unique()))
            f_values   = data[self.feature].apply(lambda x: [self.f_set.index(x)])
            self.encoder = OneHotEncoder().fit(list(f_values))
    def transform(self,data):
        if not self.f_set:
            return data
        f_values  = data[self.feature].apply(lambda x: [self.f_set.index(x)])
        f_encoded = self.encoder.transform(list(f_values)).toarray()
        f_df      = pd.DataFrame(f_encoded, columns=[self.feature+'_'+str(x) for x in self.f_set])
        new_data  = pd.concat([data.reset_index(drop=True),f_df], axis=1)
        new_data.drop(self.feature, axis=1, inplace=True)
        return new_data


def preprocess_data_X(data):
    data.fillna(0, inplace=True)
    features = [x for x in data.columns if x not in 
                    ['patientID', 'geo','segment','subgroup','combined_label', 'INTNR' ]]
    X = data[features]
    # select features
    selection1 = [  'DISTRICT', u'tribe', u'REGION_PROVINCE', u'babydoc', u'foodinsecurity', 
                    u'religion_Buddhist', u'india', u'hindu', u'religion_Hindu', 
                    u'religion_Russian/Easter', u'educ', u'Debut', u'literacy', u'christian', 
                    u'hivknow', u'ModCon', u'age', u'thrasher', u'usecondom', 
                    u'religion_Other Christia', u'religion_Muslim', u'muslim', u'lowlit', 
                    u'multpart', u'motorcycle', u'CHILDREN', u'LaborDeliv']
    selection2 = [  u'christian', u'hindu', u'REGION_PROVINCE', u'DISTRICT', u'electricity', 
                    u'age', u'tribe', u'foodinsecurity', u'EVER_HAD_SEX', u'EVER_BEEN_PREGNANT', 
                    u'CHILDREN', u'india', u'married', u'multpart', u'educ', u'literacy', 
                    u'LaborDeliv', u'babydoc', u'Debut', u'ModCon', u'usecondom', u'hivknow', 
                    u'religion_Buddhist', u'religion_Hindu', u'religion_Russian/Easter']
    selection3 = [  u'DISTRICT', u'tribe', u'REGION_PROVINCE', u'babydoc', u'india', u'educ', 
                    u'Debut', u'literacy', u'hivknow', u'ModCon', u'age', u'usecondom', 
                    u'multpart', u'CHILDREN', u'LaborDeliv', u'married']
    selected_features = selection1+selection2+selection3
    X = X[selected_features]
    return X.values

    

def preprocess_data_y(data):
    target = 'combined_label'
    data['combined_label'] = 100*data['geo'] + 10*data['segment'] + data['subgroup']
    return data['combined_label'].values

def gen_models():
    def gen_models_rf(parameters):
        models = []
        for n in parameters['n_estimators']:
            for m in parameters['max_depth']:
                models.append(RandomForestClassifier(n_estimators=n, max_depth=m, random_state=0))
        return models
    def gen_models_gbm(parameters):
        models = []
        for n in parameters['n_estimators']:
            for m in parameters['max_depth']:
                for l in parameters['learning_rate']:
                    models.append(GradientBoostingClassifier(n_estimators=n, max_depth=m, random_state=0))
        return models
    def gen_models_log(parameters):
        models = []
        for p in parameters['penalty']:
            for c in parameters['C']:
                models.append(linear_model.LogisticRegression(penalty=p, C=c))
        return models

    def gen_models_svm(parameters):
        models = []
        for c in parameters['C']:
            for k in parameters['kernel']:
                if 'gamma' in parameters:
                    for g in parameters['gamma']:
                        models.append(svm.SVC(C=c, kernel=k, gamma=g, probability=True))
                else:
                    models.append(svm.SVC(C=c, kernel=k, probability=True))

        return models

    param_gbm = { 'n_estimators':[50, 100,200],
                  'learning_rate':[0.1,1.0],
                  'max_depth':[1,2,3]}
    param_rf  = {'n_estimators':[100,200,500,800,1000,],
                  'max_depth':[1,2,4,6,8,10,13]}
    param_svc = [
                  {'C':[0.1, 1, 10, 100], 'kernel': ['linear']},
                  {'C':[0.1, 1, 10, 100], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
                ]
    param_log = {'penalty':('l1','l2'), 
                  'C':[0.1, 1, 10, 100]}
    models  = gen_models_gbm(param_gbm)
    models += gen_models_rf(param_rf)
    models += gen_models_svm(param_svc[0])
    models += gen_models_svm(param_svc[1])
    models += gen_models_log(param_log)
    selection_ind = [45, 38, 52, 31, 24, 16, 17, 30, 14, 15, 37]
    models = [models[i] for i in selection_ind]
    return models


class model_collection:
    def __init__(self, models):
        self.models = models[:]
    def fit(self, X_train, y_train):
        self.classes = np.unique(y_train)
        for i in range(len(self.models)):
            self.models[i].fit(X_train,y_train)
    def predict_proba(self, X_test):
        self.y_proba = []
        for m in self.models:
            self.y_proba.append(m.predict_proba(X_test))
        return self.y_proba
    def predict(self, X_test):
        self.y_pred = []
        for m in self.models:
            self.y_pred.append(m.predict(X_test))
        return self.y_pred
    

class combined_model:
    def __init__(self):
        self.models = gen_models()
    def fit(self, X_train, y_train):
        self.classes = np.unique(y_train)
        for i in range(len(self.models)):
            self.models[i].fit(X_train,y_train)
    def predict(self, X_test):
        y_pred      = np.array([m.predict(X_test) for m in self.models]).T
        y_proba     = [m.predict_proba(X_test) for m in self.models]
        y_proba_max = np.array([[row.max() for row in yy] for yy in y_proba]).T
        # The sum of probabilities
        y_proba_sum = np.zeros_like(y_proba[0])
        for yy in y_proba:
            y_proba_sum += yy
        y_pred_sum = [self.classes[row.argmax()] for row in y_proba_sum]
        
        prediction = np.array(y_pred_sum)
        return prediction
        
        
def azureml_main(train_data = None, test_data = None):
    features = [x for x in train_data.columns if x not in ['geo','segment','subgroup' ]]
    test_data = test_data[features]

    t = encode_categorical_feature('religion')
    t.fit(train_data)
    test_data = t.transform(test_data)
    X_test  = preprocess_data_X(test_data)

    m = pickle.load(open("./Script Bundle/model_7.1_sum_of_models.pkl",'rb'))

    y_pred = m.predict(X_test)
    
    test_data['Geo_Pred']      = y_pred/100
    test_data['Segment_Pred']  = y_pred/10%10
    test_data['Subgroup_Pred'] = y_pred%10
    
    return test_data[['patientID','Geo_Pred','Segment_Pred','Subgroup_Pred']]
    # output = pd.DataFrame()
    # output['patientID']=test_data['patientID']
    # output['Geo_Pred']      = y_pred/100
    # output['Segment_Pred']  = y_pred/10%10
    # output['Subgroup_Pred'] = y_pred%10
    # return output



In [20]:
data = pd.read_csv('../datasets/WomenHealth_Training.csv')
train_data, test_data = data.copy(), data.copy()

t = encode_categorical_feature('religion')
t.fit(train_data)
train_data = t.transform(train_data)
test_data = t.transform(test_data)
X_train = preprocess_data_X(train_data)
y_train = preprocess_data_y(train_data)
X_test  = preprocess_data_X(test_data)
y_test  = preprocess_data_y(test_data)


In [25]:
# m = combined_model()
# m.fit(X_train,y_train)
print sklearn.metrics.accuracy_score(y_test, m.predict(X_test))
import os
import pickle
print os.getcwd()
# pickle.dump(m, open('model_7.1_sum_of_models.pkl', 'wb'))

0.98750709824
/Users/Jie/Dropbox/work/side_projects/ml_compitations/women_health/src


In [26]:
# m2 = pickle.load(open('model_7.1_sum_of_models.pkl','rb'))
# sklearn.metrics.accuracy_score(y_test, m2.predict(X_test))

0.98750709823963656